# Notebook, Dimension reduction

Create a summary of Dimension reduction machine learning methods using Wine quality data as the subject.

*Classification method<br>
https://www.kaggle.com/urayukitaka/notebook-classification-method<br>
*Regression method<br>
https://www.kaggle.com/urayukitaka/notebook-regression-method

## Dimension reduction Medhod
- Factor analysis
- PCA (Principal component analysis)
- ICA (Independent component analysis)
- K-means Cluster analysis
- Hierarchical clustering
- Gaussian Mixture Model
- Isomap
- t-SNE

## Libraries

In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Visualization
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import seaborn as sns

# Data preprocessing
from sklearn.preprocessing import StandardScaler

## Dataloadin and checking

In [ ]:
# data loading
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
# dataframe
df.head()

In [ ]:
# Null values
df.isnull().sum().sum()

In [ ]:
# Data info
df.info()

## Data distribution

In [ ]:
# Sample 200
sns.pairplot(df.sample(400, random_state=10), hue="quality")

In [ ]:
# Correlation
matrix = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(matrix, vmax=1, vmin=-1, cmap="bwr", square=True)

## Data preprocessing

In [ ]:
# Variables
columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
X = df[columns]
label = df["quality"]

# Scaling
sc = StandardScaler()
sc.fit(X)

X_std = sc.fit_transform(X)

## Dimension reduction Medhod
- Factor analysis
- PCA (Principal component analysis)
- ICA (Independent component analysis)
- K-means Cluster analysis
- Hierarchical clustering
- Gaussian Mixture Model
- Isomap
- t-SNE

# Factor analysis
https://factor-analyzer.readthedocs.io/en/latest/factor_analyzer.html

In [ ]:
# Library
!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer

### Scree plot

Determine the number of features to decompose by visualizing the amount of information with a scree plot

In [ ]:
# calculate eigen values
eigen_vals = sorted(np.linalg.eigvals(pd.DataFrame(X_std).corr()), reverse=True)

# plot
plt.figure(figsize=(10,6))
plt.plot(eigen_vals, 's-')
plt.xlabel("factor")
plt.ylabel("eigenvalue")

- It can be aggregated into 5 variables.

In [ ]:
# Create instance
fa = FactorAnalyzer(n_factors=5, rotation="varimax", impute="drop")

# Fitting
fa.fit(X)
result = fa.loadings_
colnames = columns

# Visualization by heatmap
plt.figure(figsize=(10,10))
hm = sns.heatmap(result, cbar=True, annot=True, cmap='bwr', fmt=".2f", 
                 annot_kws={"size":10}, yticklabels=colnames, xticklabels=["factor1", "factor2", "factor3", "factor4", "factor5"], vmax=1, vmin=-1, center=0)
plt.xlabel("factors")
plt.ylabel("variables")

From their values, I can name their factors.<br>
factor 0 : Total Acidity<br>
factor 1 : Dioxide<br>
factor 2 : Chlirides<br>
factor 3 : Refreshing alcohol<br>
factor 4 : A little sweet and sour and rich<br>

### Plot with 1st and 2nd factor

In [ ]:
# factor dataframe
factor_df = pd.DataFrame(fa.transform(X), columns=["factor1", "factor2", "factor3", "factor4", "factor5"])
factor_df.head()

In [ ]:
# Check correlation fot quality
matrix = pd.concat([factor_df, label], axis=1).corr()
plt.figure(figsize=(10,10))
sns.heatmap(matrix, vmax=1, vmin=-1, cbar=True, annot=True, cmap='bwr', fmt=".2f",  square=True)

factor2 and factor4 have negative weak correlation.

In [ ]:
# visualization by plot
x = factor_df["factor2"]
y = factor_df["factor4"]
c = label

plt.figure(figsize=(10,8))
plt.scatter(x, y, c=c, alpha=0.7)
plt.xlabel("factor2 (Chlirides)")
plt.ylabel("factor4 (A little sweet and sour and rich)")
plt.colorbar()

# PCA (Principal component analysis)
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html?highlight=pca#sklearn.decomposition.PCA

In [ ]:
# Library
from sklearn.decomposition import PCA

# Create instance
pca = PCA(n_components=5)

In [ ]:
# Fitting
pca_result = pca.fit_transform(X)
pca_result = pd.DataFrame(pca_result, columns=("pca1", "pca2", "pca3", "pca4", "pca5"))
pca_result.head()

Visualize the coefficient of each principal component with heat map

In [ ]:
# Visualization by heatmap
plt.figure(figsize=(10,10))
hm = sns.heatmap(pca.components_.T, cbar=True, annot=True, cmap='bwr', fmt=".2f", 
                 annot_kws={"size":10}, yticklabels=colnames, xticklabels=pca_result.columns, vmax=1, vmin=-1, center=0)
plt.xlabel("pca")
plt.ylabel("variables")

Check how the quality is divided with respect to the principal component axis with a diagram

In [ ]:
# visualization by plot
x = pca_result["pca3"]
y = pca_result["pca5"]
c = label

plt.figure(figsize=(10,8))
plt.scatter(x, y, c=c, alpha=0.7)
plt.xlabel("PCA3")
plt.ylabel("PCA5")
plt.colorbar()

# ICA (Independent component analysis)
Used when normality cannot be assumed for the original data.<br>

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html?highlight=ica#sklearn.decomposition.FastICA

In [ ]:
# Library
from sklearn.decomposition import FastICA

# Create instance
ica = FastICA(n_components=5)

In [ ]:
# Fitting
ica_result = ica.fit_transform(X)
ica_result = pd.DataFrame(ica_result, columns=("ica1", "ica2", "ica3", "ica4", "ica5"))
ica_result.head()

In [ ]:
# Visualization by heatmap
plt.figure(figsize=(10,10))
hm = sns.heatmap(ica.mixing_, cbar=True, annot=True, cmap='bwr', fmt=".2f", 
                 annot_kws={"size":10}, yticklabels=colnames, center=0)
plt.xlabel("ica")
plt.ylabel("variables")

Check how the quality is divided with respect to the independent component axis

In [ ]:
# visualization by plot
x = ica_result["ica3"]
y = ica_result["ica5"]
c = label

plt.figure(figsize=(10,8))
plt.scatter(x, y, c=c, alpha=0.7)
plt.xlabel("ICA3")
plt.ylabel("ICA5")
plt.colorbar()

ICA has not been able to decompose the features of this data well.

# K-means Cluster analysis
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html?highlight=kmeans#sklearn.cluster.KMeans

First, with elbow method, cluster count can be decided.

In [ ]:
# Library
from sklearn.cluster import KMeans

# algorism
distortions = []
for i in range(1,11):
    km = KMeans(n_clusters=i, init="k-means++", n_init=10, max_iter=100, random_state=10)
    km.fit(X_std)
    distortions.append(km.inertia_)
    
# Plotting distortions
plt.figure(figsize=(10,6))
plt.plot(range(1,11), distortions, marker='o')
plt.xlabel("Number of clusters")
plt.xticks(range(1,11))
plt.ylabel("Distortion")

Elbow is between 6 and 7, may be 6 is best from distortion.

Since the quality is divided into 6, and elbow method result is 6, so I define the number of the initial cluster as 6 and execute.

In [ ]:
# Create instance
kmeans = KMeans(n_clusters=6, max_iter=30, init="k-means++", random_state=10)

In [ ]:
# Fitting
kmeans.fit(X_std)

In [ ]:
# output
cluster = kmeans.labels_

# Combine with PCA results
pca_result["cluster"] = cluster

Visualize cluster overlap in combination with PCA

In [ ]:
# visualization, cluster
x = pca_result["pca3"]
y = pca_result["pca5"]
cluster = pca_result["cluster"]

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=cluster)
plt.xlabel("PCA3")
plt.ylabel("PCA5")
plt.title("Cluster analysis result")
plt.colorbar()

Graphically compare the features of each cluster

In [ ]:
# Feature comparison of each cluster
cluster_df = pd.DataFrame(X_std, columns=X.columns)
cluster_df["cluster"] = cluster

cluster_mean = cluster_df.groupby("cluster").mean().T

cluster_std = cluster_df.groupby("cluster").std().T

In [ ]:
# Visualization
cluster_name = ["cluster1","cluster2","cluster3","cluster4","cluster5","cluster6"]
fig, ax = plt.subplots(2,3,figsize=(20,12))
plt.subplots_adjust(hspace=0.8)

for i in range(len(cluster_mean.columns)):
    if i <=2:
        ax[0,i].plot(cluster_mean.index, cluster_mean[i])
        ax[0,i].fill_between(cluster_mean.index, cluster_mean[i]+cluster_std[i], cluster_mean[i]-cluster_std[i], color="blue", alpha=0.3)
        ax[0,i].set_title(cluster_name[i])
        ax[0,i].set_xlabel("variables")
        ax[0,i].set_ylabel("Standarlized values")
        ax[0,i].set_ylim([-2,8])
        ax[0,i].tick_params(axis='x', labelrotation=90)
    else:
        ax[1,i-3].plot(cluster_mean.index, cluster_mean[i])
        ax[1,i-3].fill_between(cluster_mean.index, cluster_mean[i]+cluster_std[i], cluster_mean[i]-cluster_std[i], color="blue", alpha=0.3)
        ax[1,i-3].set_title(cluster_name[i])
        ax[1,i-3].set_xlabel("variables")
        ax[1,i-3].set_ylabel("Standarlized values")
        ax[1,i-3].set_ylim([-2,8])
        ax[1,i-3].tick_params(axis='x', labelrotation=90)

Compare quality assessment scores by cluster

In [ ]:
# create data
quality_cluster = pd.DataFrame({"cluster":cluster, "quality":df["quality"]})
mapping = {0:"cluster1", 1:"cluster2", 2:"cluster3", 3:"cluster4", 4:"cluster5", 5:"cluster6"}
quality_cluster["cluster"] = quality_cluster["cluster"].map(mapping)
quality_cluster = quality_cluster.sort_values(by="cluster")

# sample count
sample_count = pd.DataFrame({"cluster":quality_cluster["cluster"].value_counts().index,"count":quality_cluster["cluster"].value_counts()})

# merge
quality_cluster = pd.merge(quality_cluster, sample_count, on="cluster", how="left")
quality_cluster["cluster+count"] = [str(str(quality_cluster["cluster"][i]+"\n N="+str(quality_cluster["count"][i]))) for i in range(len(quality_cluster))]

# Visualization by boxplot
plt.figure(figsize=(10,6))
sns.violinplot("cluster+count", "quality", data=quality_cluster, xticklabels=cluster_name)

# Hierarchical clustering
https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html

In [ ]:
# Library
from scipy.cluster.hierarchy import dendrogram, linkage

# Create instance
h_cluster = linkage(X, method = 'average', metric = 'euclidean')

# Since the number of N is large, omitted by 100 samples
plt.figure(figsize=(25,6))
dendr = dendrogram(h_cluster, p=100, truncate_mode="lastp")
plt.xticks(fontsize=8)
print("dendrogram")

In [ ]:
# Since the number of N is large, level = 3 adjusted
plt.figure(figsize=(25,6))
dendr = dendrogram(h_cluster, p=5, truncate_mode="level")
plt.xticks(fontsize=8)
print("dendrogram")

# Gaussian Mixture Model
https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
# Library
from sklearn.mixture import GaussianMixture

# Create instance
gaussian = GaussianMixture(n_components=6, max_iter=30, random_state=10)

In [ ]:
# Fitting
gaussian.fit(X_std)

In [ ]:
# output
cluster = gaussian.predict(X_std)

# Combine with PCA results
pca_result["cluster"] = cluster

In [ ]:
# Probability
probs = gaussian.predict_proba(X_std)
cluster_name = ["cluster1","cluster2","cluster3","cluster4","cluster5","cluster6"]

pd.DataFrame(probs, columns=cluster_name).round(3).head()

In [ ]:
# visualization, cluster, with probability
size = 100*probs.max(1)**2 # Square to emphasize

x = pca_result["pca3"]
y = pca_result["pca5"]
cluster = pca_result["cluster"]

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=cluster, s=size, alpha=0.7)
plt.xlabel("PCA3")
plt.ylabel("PCA5")
plt.title("Cluster analysis result")
plt.colorbar()

### Checking AIC and BIC of n_components

In [ ]:
# Calculate each n_components
n_components = np.arange(1,21)
models = [GaussianMixture(n, covariance_type="full", random_state=10, reg_covar=0.001).fit(X_std) for n in n_components]

plt.figure(figsize=(10,6))
plt.plot(n_components, [m.bic(X_std) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X_std) for m in models], label='AIC')
plt.legend()
plt.xlabel("n_components")
plt.xticks(range(1,21))
print()

by BIC, best n_components is 7, but by AIC, need to increase more.

# Isomap

In [ ]:
# Library
from sklearn.manifold import Isomap

# Create instance
iso = Isomap(n_components=2)

In [ ]:
# Fitting
iso.fit(X_std)

In [ ]:
# output
iso_projected = iso.transform(X_std)

### Plot with quality

In [ ]:
# visualization, quality
x = iso_projected[:,0]
y = iso_projected[:,1]
qual = label

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=label, alpha=0.7)
plt.xlabel("ISO 1")
plt.ylabel("ISO 2")
plt.title("ISO map visualization")
plt.colorbar()

### plot with k-means cluster

In [ ]:
# Create instance
kmeans = KMeans(n_clusters=6, max_iter=30, init="k-means++", random_state=10)

# Fitting
kmeans.fit(X_std)

# output
cluster = kmeans.labels_

# visualization, quality
x = iso_projected[:,0]
y = iso_projected[:,1]

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=cluster, alpha=0.7)
plt.xlabel("ISO 1")
plt.ylabel("ISO 2")
plt.title("ISO map visualization")
plt.colorbar()

# t-SNE

In [ ]:
# Library
from sklearn.manifold import TSNE

# Create instance
tsne = TSNE(n_components=2, random_state=10)

In [ ]:
# Fitting
tsne.fit(X_std)

In [ ]:
# Fit transform
tsne_X = tsne.fit_transform(X_std)

In [ ]:
# visualization, quality
x = tsne_X[:,0]
y = tsne_X[:,1]
qual = label

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=label, alpha=0.7)
plt.xlabel("tsne 1")
plt.ylabel("tsne 2")
plt.title("t-SNE map visualization")
plt.colorbar()

### plot with k-means cluster

In [ ]:
# Create instance
kmeans = KMeans(n_clusters=6, max_iter=30, init="k-means++", random_state=10)

# Fitting
kmeans.fit(X_std)

# output
cluster = kmeans.labels_

# visualization, quality
x = tsne_X[:,0]
y = tsne_X[:,1]

plt.figure(figsize=(10,6))
plt.scatter(x,y,c=cluster, alpha=0.7)
plt.xlabel("tsne 1")
plt.ylabel("tsne 2")
plt.title("t-SNE map visualization")
plt.colorbar()